In [1]:
import numpy as np
import pandas as pd

root_path = '/home/erie-dvrk/force_sensor_particle_filter/cal_methods/handeye/MTdata'
rotation_path = root_path + '/4.28Rotation.txt'
toolTip_path  = root_path + '/4.28ToolTip.txt'
ati_path = root_path + '/4.28ATI.txt'

In [2]:
def cal_trans(tip1, tip2, mat1, mat2):
    mat2 = np.linalg.inv(mat2)
    
    # translation vector in microntracker's frame
    transInTracker = tip2 - tip1
    
    return mat2.dot(mat1.dot(transInTracker))

In [3]:
# calculate handeye transformation
rot_f = pd.read_csv(rotation_path, sep='\t')
tip_f = pd.read_csv(toolTip_path, sep='\t')

tracker2markc = np.array(rot_f[[col for col in rot_f if col.startswith('ECM_Camera_Surface')]].values).reshape(3, 3) # rotation from tracker to camera marker
tracker2markb = np.array(rot_f[[col for col in rot_f if col.startswith('PSM_Base')]].values).reshape(3, 3)           # rotation from tracker to PSM base marker
tipc = tip_f[[col for col in tip_f if col.startswith('ECM tip')]].values[0]    # ECM marker tip
tipb = tip_f[[col for col in tip_f if col.startswith('PSM tip')]].values[0]    # PSM marker tip

# transform between markers and desired frames
cam2markc = np.array([0, -1, 0, -1, 0, 0, 0, 0, -1]).reshape(3,3)
markb2base = np.array([0, 0, -1, 1, 0, 0, 0, -1, 0]).reshape(3,3)

tracker2base = np.dot(tracker2markb, markb2base)
tracker2cam = np.dot(tracker2markc, np.linalg.inv(cam2markc))

print("Rotation")
print(np.dot(np.linalg.inv(tracker2cam), tracker2base))
print("Translation")
print(np.dot(np.linalg.inv(tracker2cam), (tipb-tipc)))

Rotation
[[-0.88481758 -0.07000301 -0.46066689]
 [-0.07601656  0.99671513 -0.00637037]
 [ 0.45985718  0.02936518 -0.88729935]]
Translation
[-90.99102688 -20.08748283  98.0965937 ]


In [4]:
# ATI sensor alignment
ati_f = pd.read_csv(ati_path, sep='\t')
baseToMarker = np.array([0, -1, 0, 0, 0, -1, 1, 0, 0]).reshape(3, 3)
base_rot = np.array(ati_f[[col for col in ati_f if col.startswith('PSM_Base')]].values).reshape(3, 3)
ati_rot = np.array(ati_f[[col for col in ati_f if col.startswith('ATI')]].values).reshape(3, 3)
print('Sensor alignment')
print(np.matmul(np.matmul(baseToMarker, np.matmul(np.linalg.inv(ati_rot), base_rot)), np.linalg.inv(baseToMarker)))

Sensor alignment
[[ 0.9970722  -0.07134282 -0.0240723 ]
 [ 0.07110851  0.99692001 -0.01121118]
 [ 0.02514205  0.00951939  0.99919272]]
